In [1]:
# Import needed packages
import torch
import torch.nn.functional as F
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import os
device = torch.device('cpu')

In [2]:
# Setup networks to compare
networks = ['subcarriers25', 'subcarriers50', 'subcarriers75', 'base']
labels = [25, 50, 75, 100]
labels = ['Benchmark [1] | 100 subcarrirers'] + ['Proposed | {} subcarrirers'.format(l) for l in labels]
scenario_name = 'subcarriers'

In [3]:
# Setup data
raw_data = {'benchmark': torch.load('output/losses/benchmark_losses.pt', map_location='cpu')} \
    | {n:torch.load('output/losses/{}_losses.pt'.format(n), map_location='cpu') for n in networks}
data = {}
for key, value in raw_data.items():
    data[key] = {}
    data[key]['training_distances'] = [x['distance'].mean() for x in value['training_losses']]
    data[key]['validation_distances'] = [x['distance'].mean() for x in value['validation_losses']]
    time_intervals = value['times']
    current_time = 0
    time = []
    for interval in time_intervals:
        current_time += interval # in ms
        time.append(current_time/60000) # in min
    data[key]['time'] = np.array(time)

In [4]:
# Plot errors
plt.close()
axis = [0, None, 0, 70]
epochs = [i for i in range(200)]

# Plot benchmark
plt.plot(epochs, data['benchmark']['validation_distances'], color='black', linestyle='dotted')
plt.savefig('output/plots/{}/benchmark_abs_error.png'.format(scenario_name))

# Plot networks
for network in networks:
    plt.plot(epochs, data[network]['validation_distances'])
plt.axis(axis)
plt.legend(labels)
plt.xlabel('Epoch')
plt.ylabel('Average distance from real position (mm)')
plt.title('Error')
plt.grid()
plt.savefig('output/plots/{}/{}_all_errors.png'.format(scenario_name, scenario_name))

In [5]:
# Plot errors with time
plt.close()
axis = [0, None, 0, 70]

# Plot benchmark
plt.plot(data['benchmark']['time'], data['benchmark']['validation_distances'], color='black', linestyle='dotted')
plt.savefig('output/plots/{}/benchmark_abs_error(time).png'.format(scenario_name))

# Plot networks
for network in networks:
    plt.plot(data[network]['time'], data[network]['validation_distances'])
plt.axis(axis)
plt.legend(labels)
plt.xlabel('Time (min)')
plt.ylabel('Average distance from real position (mm)')
plt.title('Error')
plt.grid()
plt.savefig('output/plots/{}/{}_all_errors(time).png'.format(scenario_name, scenario_name))

In [ ]:
axis = [0, None, 0, 70]
# Plot separetely
plt.close()
# plt.plot([i for i in range(150)], benchmark_losses['abs_training_losses'], color='red')
plt.plot([i for i in range(150)], benchmark_losses['validation_distances'], color='black', linestyle='dotted')
# plt.axis([None, None, 0, 350])
# plt.legend(['Training losses', 'Validation Losses'])
plt.savefig('output/plots/{}/benchmark_abs_error.png'.format(scenario_name))
epochs = [i for i in range(200)]
for network, losses in n_dict.items():
    # Plot absolute errors
    # plt.close()
    # plt.plot(epochs, losses['abs_training_losses'], color='red')
    plt.plot(epochs, losses['validation_distances'])
    plt.axis(axis)
    if network == 'base':
        print(min(losses['validation_distances']))
    # plt.legend(['Training losses', 'Validation losses'])
    # plt.savefig('output/plots/{}/{}_abs_error.png'.format(scenario_name, network))
plt.legend(labels)
plt.xlabel('Epoch')
plt.ylabel('Average distance from real position (mm)')
plt.title('Error')
plt.grid()
plt.savefig('output/plots/{}/{}_all_errors.png'.format(scenario_name, scenario_name))

plt.close()
plt.plot([i for i in range(150)], benchmark_losses['validation_distances'], color='black', linestyle='dotted')
plt.plot([i for i in range(200)], n_dict[networks[-1]]['abs_training_losses'], color='red')
plt.plot([i for i in range(200)], n_dict[networks[-1]]['validation_distances'], color='blue')
plt.axis([None, None, 0, 50])
plt.legend(['Benchmark [1]', 'Training losses', 'Validation Losses'])
plt.xlabel('Epoch')
plt.ylabel('Average distance from real position (mm)')
plt.title('Error')
plt.grid()
plt.savefig('output/plots/{}/base_abs_error.png'.format(scenario_name))


In [ ]:
# import torch.nn.functional as F
# _, validation, test = create_dataloader()
# # Run tests
# print('Running benchmark...')
# model = import_module('.' + 'benchmark', package='src.networks').Network()
# model_path = os.path.join('output', 'models', 'benchmark' + '.pt')
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model.to(device)
# model.eval()
# benchmark_distances = []
# with torch.no_grad():
#     for sample, label in test.loader:
#         sample, label = sample.to(device), label.to(device)

#         output = model(sample) # (1,2)
#         benchmark_distances.append(F.pairwise_distance(output, label).item())
# print('Running networks...')
# for network in networks:
#     model = import_module('.' + network, package='src.networks').Network()
#     model_path = os.path.join('output', 'models', network + '.pt')
#     device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#     model.to(device)
#     model.eval()
#     distances = []
#     with torch.no_grad():
#         for sample, label in test.loader:
#             sample, label = sample.to(device), label.to(device)

#             output = model(sample) # (1,2)
#             distances.append(F.pairwise_distance(output, label).item())
#     n_dict[network]['test_distances'] = distances

# print('Plotting...')
# Plot histograms
plt.close()
plt.xlim([0,150])
# Plot benchmark
plt.hist(benchmark_losses['validation_distances'], bins=100, density=True, histtype='step', color='black', linestyle='dotted')
for network, losses in n_dict.items():    
    distances = losses['validation_distances']
    plt.hist(distances, bins=100, density=True, histtype='step')

plt.legend(labels)
plt.xlabel('Average distance from real position (mm)')
plt.ylabel('Density')
plt.title('Histogram')
plt.savefig('output/plots/{}/{}_hist.png'.format(scenario_name, scenario_name))

# Plot histogram with subplots
# fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2)

# ax1.axis([0,150, None, None])
# ax1.hist(benchmark_losses['validation_distances'], bins=75, density=True, histtype='step', color='red')
# ax1.hist(n_dict[networks[0]]['validation_distances'], bins=100, density=True)
# ax1.set_title(labels[0])

# ax2.axis([0,150, None, None])
# ax2.hist(benchmark_losses['validation_distances'], bins=75, density=True, histtype='step', color='red')
# ax2.hist(n_dict[networks[1]]['validation_distances'], bins=100, density=True)
# ax2.set_title(labels[1])

# ax3.axis([0,150, None, None])
# ax3.hist(benchmark_losses['validation_distances'], bins=75, density=True, histtype='step', color='red')
# ax3.hist(n_dict[networks[2]]['validation_distances'], bins=100, density=True)
# ax3.set_title(labels[2])

# ax4.axis([0,150, None, None])
# ax4.hist(benchmark_losses['validation_distances'], bins=75, density=True, histtype='step', color='red')
# ax4.hist(n_dict[networks[3]]['validation_distances'], bins=100, density=True)
# ax4.set_title(labels[3])

# for ax in fig.get_axes():
#     ax.label_outer()

# fig.savefig('output/plots/{}/{}_hist_subplot.png'.format(scenario_name, scenario_name))



In [ ]:
# from src.networks.base_50sc import Network
# _, test_loader, _, test_size = create_dataloader()
# model = Network()
# state_dict = torch.load('output/models/base50sc.pt')
# model.load_state_dict(state_dict)
# model.eval()
# distances = torch.Tensor()
# with torch.no_grad():
#     for samples, labels in test_loader:
    
#         output = model(samples)

#         # distances = torch.cat((distances, F.l1_loss(output, labels, reduction='none'), dim=)
#         distances = torch.cat((distances, F.pairwise_distance(output, labels)))
#         print(distances.shape)

# # %%
# array = distances.numpy()
# plt.close()
# plt.hist(array, bins=50, density=True)
# plt.savefig('hist.png')